In [9]:
import os
from glob import glob
from pathlib import Path
import shutil
import sys
import time

In [11]:
date = time.strftime('%Y-%m-%d')

In [6]:
def get_all_files(card_path):
    all_files = []
    for root, dirs, files in os.walk(card_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files

In [4]:
# get paths of SD cards connected
if sys.platform == 'darwin':
    connected_sd_cards = [Path(card) for card in glob('/Volumes/Untitled*/')]
elif sys.platform == 'win32':
    connected_sd_cards = [Path(card) for card in glob('E:/')]
elif sys.platform == 'linux':
    connected_sd_cards = [Path(card) for card in glob('/media/')]

In [7]:
card_id = {}
cam_id = 0
for card in connected_sd_cards:
    if any(['.360' in file for file in get_all_files(card)]):
        card_id[card] = '360'
    elif any(['.mp4' in file for file in get_all_files(card)]):
        cam+=1
        card_id[card] = f'cam{cam}'
    elif any(['TRACK' in file for file in get_all_files(card)]):
        card_id[card] = 'microphones'
    else:
        print(f'Unknown card: {card} contains the following files:')
        print(get_all_files(card))

In [12]:
# func for getting the creation date of a file
def get_creation_date(path_to_file):
    return time.strftime('%Y-%m-%d', time.gmtime(os.path.getctime(path_to_file)))